## Import

In [1]:
import pandas as pd

## Parse String to Number

In [2]:
## Taken from https://medium.com/@Doug-Creates/convert-string-to-float-or-int-in-python-0d93ce89d7b1

def parse_string_to_number(input_string) :
    try:
        # Attempt to convert the string to an int
        return int(input_string)
    except ValueError:
        # If int conversion fails, attempt to convert to float
        try:
            return float(input_string)
        except ValueError:
            # If both conversions fail, return an error message
            return 'Error: Cannot convert to int or float'

## Read file

In [3]:
file = open("out.txt", "r")
text = file.read()
lines = text.split('\n')
no_lines = len(lines)

## Functions for parsing lines

In [4]:
def get_n_from_line(ln) :
    index1 = ln.find('=') + 2
    index2 = ln.find('---') - 1
    return int(ln[index1:index2])
    
def get_attr_from_line(ln) :
    colon_index = ln.find(':')
    label = ln[:colon_index-1]
    rem = ln[colon_index+2:]
    space_index = rem.find(' ')
    val = rem[:space_index]
    unit = rem[space_index+1:]
    return label, unit, parse_string_to_number(val)

## Parse outputs

In [5]:
# Attributes for first n
i = 1
col_names = ['n']
col_types = ['int']
data = [[get_n_from_line(lines[0])]]
while lines[i].find("---") == -1 :
    label, unit, val = get_attr_from_line(lines[i])
    col_names = col_names + [f"{label} ({unit})"]
    data = data + [[val]]
    col_type = 'float' if unit == 'ms' else 'int'
    col_types.append(col_type)
    i += 1
no_attrs = i-1

# Attributes for remaining n
while i <= no_lines :
    if lines[i] == '' :
        break
    n = get_n_from_line(lines[i])
    data[0].append(parse_string_to_number(n))
    for j in range(1, no_attrs+1) :
        _, _, val = get_attr_from_line(lines[i+j])
        data[j].append(val)
    i += no_attrs+1

## Create dataframe

In [6]:
df = pd.DataFrame(data, index=col_names).T
for c, t in zip(col_names, col_types) :
    df[c] = df[c].astype(t)

## Display dataframe

In [7]:
df

,n,Pre-comm (bytes),Fixed time (ms),Fixed comm (bytes)
0,4,951543,4624.559,1504537
1,5,951543,4538.240,1504539
2,6,951543,4561.266,1504543
3,7,951543,4512.719,1504551
4,8,951543,4624.311,1504567


## Save dataframe to CSV

In [8]:
df.to_csv("out.csv", index=False)